In [1]:
import requests
import csv
import time
import pandas as pd
from osm_script import osm_extractor_groups, fetch_osm_region
from reverse_geocode import geocode_lat_lon
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [3]:
#Several APIs available 

main_overpass_api = "https://overpass-api.de/api/interpreter"
lz4_overpass_api='https://lz4.overpass-api.de/api/interpreter'
osm_overpass_api='https://overpass.openstreetmap.ru/api/interpreter'

BUNDES_GROUPS = [
    ["Baden-Württemberg"],
    ["Bayern"],
    ["Berlin", "Bremen"],
    ["Brandenburg"],
    ["Hamburg"],
    ["Hessen","Mecklenburg-Vorpommern","Rheinland-Pfalz"],
    ["Niedersachsen"],
    ["Nordrhein-Westfalen"],
    ["Saarland","Sachsen","Sachsen-Anhalt", "Schleswig-Holstein","Thüringen"]
]

In [4]:
df_raw=osm_extractor_groups(BUNDES_GROUPS)

In [23]:
len(df_raw)

4604

In [ ]:
#it seems like empty cells contain empty string, therefore it needs to be replaced with NA values 
df_raw = df_raw.replace(r'^\s*$', pd.NA, regex=True)

In [25]:
df_raw.sample(10) # raw results

,osm_id,region,type,name,street,housenumber,postcode,city,lat,lon,phone,email,website,amenity
3274,725425698,Nordrhein-Westfalen,way,AWO KiTa Blaues Haus,Linsellesstraße,,47877,Willich,51.233811,6.531840,+49 2154 8847450,NaN,NaN,kindergarten
2694,11171920844,Nordrhein-Westfalen,node,zentrum plus - Außenstelle Ludenberg,Blanckertzstraße,26,40629.0,Düsseldorf,51.255742,6.864778,NaN,NaN,NaN,social_facility
1978,11235426039,Niedersachsen,node,Arbeiterwohlfahrt Lüneburg,St.-Stephanus-Passage,15,21337,Lüneburg,53.239748,10.435249,NaN,NaN,https://awo-lueneburg.de/,social_centre
16,1608718753,Baden-Württemberg,node,AWO-Haus Epfelbach,Eschelbronner Straße,,74925,Epfenbach,49.336563,8.907754,NaN,NaN,https://www.awo-rhein-neckar.de/,arts_centre
4040,93338561,Sachsen-Anhalt,way,Kinder- und Jugendhilfeeinrichtung „Haus Achte...,Bertolt-Brecht-Straße,40b,39218.0,Schönebeck (Elbe),52.016427,11.725454,+49 3928 69835,haus-achterbahn@awo-slk.de,https://www.awo-slk.de/seite/321206/kinder-und...,social_facility
4424,129216703,Thüringen,way,Servicewohnen AWO AJS gGmbH,Krämpferufer,7,99084.0,Erfurt,50.981082,11.038167,+49 361 6003815,schneider@awo-thueringen.de,NaN,nursing_home
236,439365332,Baden-Württemberg,way,Altenzentrum Königsbronn,Brenzstraße,17,89551.0,Königsbronn,48.736069,10.118724,+497328 92390-0,NaN,http://awo-heidenheim.de/altenzentren/altenzen...,social_facility
46,4853952122,Baden-Württemberg,node,AWO Kindervilla,Bahnhofstraße,19,68526.0,Ladenburg,49.472006,8.604160,+49 6203 1079229,NaN,http://www.awo-rhein-neckar.de/kinderbetreuung...,kindergarten
1228,237400422,Brandenburg,way,Arbeiterwohlfahrt,Hartmannsdorfer Straße,15,15907,Lübben (Spreewald),51.948635,13.884383,NaN,NaN,NaN,social_facility
1494,302673393,Hessen,way,Kindergarten am Zwingel,Am Zwingel,1,35683.0,Dillenburg,50.740137,8.286755,NaN,NaN,NaN,kindergarten


In [18]:
df_raw.region.value_counts()

region
Nordrhein-Westfalen       1236
Bayern                     679
Niedersachsen              513
Thüringen                  300
Baden-Württemberg          299
Sachsen                    281
Hessen                     237
Schleswig-Holstein         190
Brandenburg                163
Mecklenburg-Vorpommern     156
Sachsen-Anhalt             142
Saarland                   121
Rheinland-Pfalz             94
Berlin                      84
Bremen                      69
Hamburg                     40
Name: count, dtype: int64

In [24]:
df_raw.isnull().sum()

osm_id            0
region            0
type              0
name            413
street           16
housenumber      47
postcode         18
city              0
lat               0
lon               0
phone          3138
email          3827
website        2846
amenity        1308
dtype: int64

Next: 
    - removing findings which are not AWO facilities 
    - comparing with current Einrichtungsdatenbank 
    - comaring with domains .xlsx 
    - ...


In [4]:
#if need to restart kernel, import the latest file 
df_raw = pd.read_csv("awo_20250926-135243_osm_raw.csv")
df_raw.shape

(4604, 14)

In [23]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4604 entries, 0 to 4603
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   osm_id       4604 non-null   int64  
 1   region       4604 non-null   object 
 2   type         4604 non-null   object 
 3   name         4191 non-null   object 
 4   street       4588 non-null   object 
 5   housenumber  4557 non-null   object 
 6   postcode     4586 non-null   object 
 7   city         4604 non-null   object 
 8   lat          4604 non-null   float64
 9   lon          4604 non-null   float64
 10  phone        1466 non-null   object 
 11  email        777 non-null    object 
 12  website      1758 non-null   object 
 13  amenity      3296 non-null   object 
dtypes: float64(2), int64(1), object(11)
memory usage: 503.7+ KB


In [ ]:
mask_empty_address = df_raw['city'].isna()
dict_empty_address = df_raw.loc[mask_empty_address].apply(lambda row:geocode_lat_lon(row['lat'], row['lon']), axis=1)


NameError: name 'dict_emtpy_address' is not defined

In [17]:
lat=df_raw.loc[2446, 'lat']
lon = df_raw.loc[2446, 'lon']
geocode_lat_lon(lat, lon)


{'postcode': '32120',
 'city': 'Hiddenhausen',
 'street': 'Unter der Weide',
 'housenumber': '14'}

In [22]:
add_addresses = pd.DataFrame(list(dict_empty_address), index=dict_empty_address.index)
df_raw.loc[mask_empty_address, add_addresses.columns] = add_addresses

/tmp/ipykernel_76482/3849740406.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['76275' '79331' '73525' ... '37327' '99427' '36448']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_raw.loc[mask_empty_address, add_addresses.columns] = add_addresses


In [ ]:
pattern_upper = r"\bAWO\b" # remove false positives WIP
has_word = df['name'].str.contains(pattern_word, case=False, na=False)
#has_any = big['name'].str.contains("awo", case=False, na=False)
mask = has_word|(~has_any)
df_filtered = df.loc[has_word].reset_index(drop=True)

In [22]:
name_datetime = time.strftime("%Y%m%d-%H%M%S")
df_raw.to_csv(f"awo_{name_datetime}_osm_raw.csv", index=False, encoding='utf-8')

In [ ]:
#domains1=domains.assign(InOsm=domains.domain.isin(df.website).astype(int))